In [ ]:
# Install our codebase 
# !pip install ECHO_modules >&/dev/null;
!pip install git+https://github.com/edgi-govdata-archiving/ECHO_modules@neighborhoods >&/dev/null;
!pip install geopandas >&/dev/null;

In [ ]:
from ECHO_modules.get_data import get_echo_data
from ECHO_modules.utilities import show_region_type_widget, \
    show_state_widget, show_year_range_widget
from ECHO_modules.rsei_utilities import show_rsei_pick_region_widget

region_type_widget = show_region_type_widget(region_types=('City', 'County', 'State', 'Zip Code', 'Neighborhood'), 
                                             default_value='City' )
state_widget = None
# display( region_type_widget )
print('(The State will be ignored for Zip Code and Neighborhood regions.)')
state_widget = show_state_widget()

## Select the regions to look for
For Neighborhoods, only rectangles are currently supported


In [ ]:
from ECHO_modules.utilities import polygon_map

region_widget = None
region_type = region_type_widget.value
if ( region_type == 'Neighborhood' ):
    (map,shapes) = polygon_map()
    display(map)
elif ( region_type != 'State' ):
    region_widget = show_rsei_pick_region_widget( type=region_type,
                                           state_widget=state_widget )

## See facilities for the chosen regions

In [ ]:
from ECHO_modules.rsei_utilities import get_rsei_facilities

state = state_widget.value if state_widget is not None else None
regions_selected = None
if ( region_type == 'Zip Code' ):
    regions_selected = str(region_widget.value)
elif ( region_type == 'Neighborhood' ):
    regions_selected = shapes.pop()
elif ( region_type != 'State' ):
    regions_selected = region_widget.value
    
columns = '"FacilityName", "FacilityID", "FacilityNumber", "FRSID", "Latitude", "Longitude", "Street",'
columns += '"City", "County", "State", "ZIPCode", "StandardizedParentCompany"'

fac_df = get_rsei_facilities(state=state, region_type=region_type, regions_selected=regions_selected, 
                             rsei_type='facility', columns=columns)
# If the columns aren't specified, all columns are returned ("select * from ...")
# fac_df = get_rsei_facilities(state=state, region_type=region_type, regions_selected=regions_selected, 
#                              rsei_type='facility')
fac_df

In [ ]:
from ECHO_modules.utilities import mapper

map_of_facilities = mapper(fac_df, no_text=False, lat_field='Latitude', long_field='Longitude', name_field='FacilityName')
display(map_of_facilities)

In [ ]:
from ECHO_modules.utilities import show_year_range_widget
year_range = show_year_range_widget()

In [ ]:
from ECHO_modules.rsei_utilities import get_submissions_by_facilities

columns = '"SubmissionNumber", "FacilityNumber", "ChemicalNumber", "SubmissionYear", "OneTimeReleaseQty", "TradeSecretInd"'

# sub_df = get_submissions_by_facilities(facilities=None, columns='*', years=year_range.value)
sub_df = get_submissions_by_facilities(facilities=fac_df['FacilityNumber'], columns=columns, years=year_range.value)
sub_df

## Add the chemicals to the submissions

In [ ]:
from ECHO_modules.rsei_utilities import add_chemical_to_submissions

# columns = '"Chemical", "RfCInhale", "RfDOral"'
columns = '*'
sub1_df = add_chemical_to_submissions(submissions=sub_df, chemical_columns=columns)
columns = ["SubmissionNumber", "ChemicalNumber", "Chemical", "RfCInhale"]
sub1_df[columns]

## See offsite facilities for the chosen region

In [ ]:
from ECHO_modules.rsei_utilities import get_rsei_facilities

columns = '"Name", "OffsiteID", "FacilityNumber", "TRIFID", "FRSID", "Latitude", "Longitude", "Street",'
columns += '"City", "State", "ZIPCode"'

off_df = get_rsei_facilities(state=state, region_type=region_type, regions_selected=regions_selected, 
                             rsei_type='offsite', columns=columns)
off_df

In [ ]:

from ECHO_modules.utilities import mapper

to_map = off_df.dropna(subset=['Latitude', 'Longitude'])
map_of_facilities = mapper(to_map, no_text=False, lat_field='Latitude', 
                           long_field='Longitude', name_field='Name')
display(map_of_facilities)

In [ ]:

# All the releases where media = 1 (I think that's direct air releases) 
rsql = 'select * from "releases_data_rsei_v2312" where "Media" <= 2;' 
get_echo_data(rsql)
# All the releases above a certain weight 
rsql = 'select * from "releases_data_rsei_v2312" where "PoundsReleased" > 100000;' 
releases = get_echo_data(rsql)

In [ ]:
len(releases)

In [ ]:
# All the releases where media = 1 (I think that's direct air releases) 
media_sql = 'select "Media", "MediaText" from "media_data_rsei_v2312";' 
media_types = get_echo_data(media_sql)
media_types

In [ ]:
# Get Exxon facilities 
rsql = 'select * from "facility_data_rsei_v2312" where "StandardizedParentCompany" like \'%EXXON%\';' 
facs = get_echo_data(rsql) 
# Get their submissions 
these_fac_numbers = list(facs["FacilityNumber"].unique()) 
rsql = 'select * from "submissions_data_rsei_v2312" where "FacilityNumber" in ({});'.format(','.join([str(fac) for fac in these_fac_numbers])) 
# You shouldn't do SQL like this but I'm being quick 
subs = get_echo_data(rsql) 

# Use these submission numbers to get releases 
# Ok, actually there are too many submissions (>20,000) to easily get all the Exxon releases from the database. 
# An enterprising SQL writer could do this with some joins, I bet! No time right now for me though 
# But this is the general idea.... 
these_submission_numbers = list(subs["SubmissionNumber"].unique())[0:50] 
# Just do the first 50 as a test 
rsql = 'select * from "releases_data_rsei_v2312" where "SubmissionNumber" in ({});'.format(','.join([str(fac) for fac in these_submission_numbers])) 
res = get_echo_data(rsql) 
res